In [1]:
import xarray
import numpy as np
import pandas as pd

emission_CO2_netcdf_AM2 = xarray.open_dataset("D:/emission_CO2_AM1_3tahun_v2.nc", engine="netcdf4")
gfrac = xarray.open_dataset("D:/kerja/asisten riset/vol/milkunC/achaidir/IMAGE PBL/SSP2/GFRAC_NEW.nc", engine="netcdf4")
garea = xarray.open_dataset("D:/kerja/asisten riset/vol/milkunarc/cadlan/Analysis1_slidingwindows/GAREA.NC", engine="netcdf4")
gfrac_list = gfrac.coords['NGFBFC'].data.tolist()
gfrac_ngfbfc = [element.strip() for element in gfrac.coords['NGFBFC'].data.astype('str').tolist()]

In [2]:
garea_7020 = garea.isel(time=slice(0, 3))
garea_ha = np.nan_to_num(np.multiply(garea_7020['GAREA'].to_numpy(), 100))

gfrac_7020 = gfrac.isel(time=slice(0, 3)) 
gfrac_area = np.zeros((3, 17, 2160, 4320), dtype="float32")

In [3]:
for n in range(3):
    for i in range(17):
        gfrac_area[n][i] = np.nan_to_num(np.multiply(gfrac_7020['GFRAC_new'].isel(time=n, NGFBFC=i), garea_ha[n]))

gfrac_list = gfrac.coords['NGFBFC'].data.tolist()

gfrac_area_netcdf = xarray.Dataset({
    "GFRAC_combined":(["time","NGFBFC","latitude", "longitude"], gfrac_area)
},coords={
        "time": pd.date_range(start='1970-01-01', end='1980-01-01', freq='5YS'),
        "NGFBFC" : gfrac_list,
        "latitude": gfrac.coords["latitude"].to_numpy(),
        "longitude": gfrac.coords["longitude"].to_numpy()
})

In [5]:
np.where(gfrac_area[0][1] < 0)

(array([], dtype=int64), array([], dtype=int64))

## luc crops

In [5]:
x = [-51.706, -55.974643, 101.457, 109.38, -96.6289, 48.966]
y = [-22.698, -21.208333, 31.4420, -7.11, 18.549639, -18.07]

n = 4
gfrac['GFRAC_new'].isel(time=slice(0, 3)).sel(latitude=y[n], longitude=x[n], method='nearest').to_pandas()

NGFBFC,b'grass ',b'Wheat ',b'Rice ',b'Maize ',b'Tropical cereals ',b'Other temperate cereals ',b'Pulses ',b'Soybeans ',b'Temperate oil crops ',b'Tropical oil crops ',b'Temperate roots & tubers ',b'Tropical roots & tubers ',b'Sugar crops ',b'Oil & palm fruit ',b'Vegetables & fruits ',b'Other non-food & luxury & spices ',b'Plant based fibres '
time,,,,,,,,,,,,,,,,,
1970-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1975-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1980-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [18]:
akumulasi = np.zeros((1, 17, 2160, 4320))

for time in range(3):
    for i, crop in enumerate(gfrac_list):
        temp = np.where(gfrac['GFRAC_new'].isel(time=time).sel(NGFBFC=crop).to_numpy() > 0, gfrac['GFRAC_new'].isel(time=time).sel(NGFBFC=crop).to_numpy(), 0)
        akumulasi[0][i] += np.nan_to_num(temp) 
        akumulasi[0][i] = np.nan_to_num(akumulasi[0][i])

In [19]:
np.where(akumulasi[0] < 0)

(array([], dtype=int64), array([], dtype=int64), array([], dtype=int64))

In [24]:
akumulasi_netcdf = xarray.Dataset({
    "akumulasi":(["time", "NGFBFC","latitude", "longitude"], akumulasi)
},coords={
        "time": pd.to_datetime(["2020-01-01"]),
        "NGFBFC" : gfrac_ngfbfc,
        "latitude": gfrac.coords["latitude"].to_numpy(),
        "longitude": gfrac.coords["longitude"].to_numpy()
})

In [7]:
akumulasi_netcdf['akumulasi'].isel(time=0, NGFBFC=0).sel(latitude=y[n], longitude=x[n], method='nearest').values

array(0.)

In [25]:
#berapa tahun kemunculan
kemunculan = np.zeros((1, 17, 2160, 4320))

for time in range(3):
    for i, crop in enumerate(gfrac_list):
        kemunculan[0][i] += np.where(gfrac['GFRAC_new'].isel(time=time).sel(NGFBFC=crop).to_numpy() > 0, 1, 0)

In [26]:
np.where(kemunculan < 0)

(array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64))

In [8]:
kemunculan_netcdf = xarray.Dataset({
    "kemunculan":(["time", "NGFBFC","latitude", "longitude"], kemunculan)
},coords={
        "time": pd.to_datetime(["2020-01-01"]),
        "NGFBFC" : gfrac_ngfbfc,
        "latitude": gfrac.coords["latitude"].to_numpy(),
        "longitude": gfrac.coords["longitude"].to_numpy()
})

In [10]:
kemunculan_netcdf['kemunculan'].isel(time=0, NGFBFC=slice(0, 2)).sel(latitude=y[n], longitude=x[n], method='nearest').values

array([0., 0.])

In [27]:
proporsi = np.zeros((17, 1, 2160, 4320))

for i, crop in enumerate(gfrac_list):
    temp = np.divide(akumulasi[0][i], kemunculan[0][i])
    proporsi[i][0] = np.nan_to_num(temp)

C:\Users\HEFRYA~1\AppData\Local\Temp/ipykernel_2736/3424421911.py:4: RuntimeWarning: invalid value encountered in divide
  temp = np.divide(akumulasi[0][i], kemunculan[0][i])


In [28]:
np.where(proporsi < 0)

(array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64),
 array([], dtype=int64))

In [31]:
proporsi_netcdf = xarray.Dataset({
    "proporsi":(["NGFBFC", "time","latitude", "longitude"], proporsi)
},coords={
        "time": pd.to_datetime(["2020-01-01"]),
        "NGFBFC" : gfrac_ngfbfc,
        "latitude": gfrac.coords["latitude"].to_numpy(),
        "longitude": gfrac.coords["longitude"].to_numpy()
})
# proporsi_netcdf.to_netcdf("D:/kerja/asisten riset/code v/step 3/GbGlct/dataset/AM3/proporsi_AM3.NC", mode="w", format="NETCDF4")

In [14]:
proporsi2_netcdf = xarray.Dataset(
coords={
        "time": pd.to_datetime(["2020-01-01"]),
        "latitude": gfrac.coords["latitude"].to_numpy(),
        "longitude": gfrac.coords["longitude"].to_numpy()
    })

coords = ("time", "latitude", "longitude")
data_vars = {
    ngfbc: (coords, proporsi[i]) for i, ngfbc in enumerate(gfrac_ngfbfc)
}
proporsi2_netcdf = proporsi2_netcdf.assign(data_vars)
proporsi2_netcdf.to_netcdf("D:/kerja/asisten riset/code v/step 3/GbGlct/dataset/AM3/proporsi2_AM3.NC", mode="w", format="NETCDF4")

In [33]:
n = 5
proporsi_netcdf['proporsi'].isel(time=0).sel(latitude=y[n], longitude=x[n], method='nearest').sum(dim='NGFBFC').to_pandas()

array(0.99999999)

In [32]:
def divide(arr1, arr2):
    return np.divide(arr1, arr2)

def multiple(arr1, arr2):
    return np.multiply(arr1, arr2)

In [35]:
#2C Distribusi emisi sesuai delta
luc_prop_crop = np.zeros((17, 1, 2160, 4320), dtype="float32")

for i, crop in enumerate(gfrac_list):
    luc_prop_crop[i][0] = np.nan_to_num(multiple(emission_CO2_netcdf_AM2['LUC Crops Emission'].isel(time=0).to_numpy(), proporsi[i][0]))

In [36]:
luccrop_emisi_netcdf = xarray.Dataset({
    "luc_crop":(["NGFBFC", "time","latitude", "longitude"], luc_prop_crop)
},coords={
        "time": pd.to_datetime(["2020-01-01"]),
        "NGFBFC" : gfrac_ngfbfc,
        "latitude": gfrac.coords["latitude"].to_numpy(),
        "longitude": gfrac.coords["longitude"].to_numpy()
})

In [37]:
emission_CO2_netcdf_AM2['LUC Crops Emission'].isel(time=0).sel(latitude=y[n], longitude=x[n], method='nearest').to_pandas()

array(0., dtype=float32)

In [19]:
luccrop_emisi_netcdf['luc_crop'].isel(time=0).sel(latitude=y[n], longitude=x[n], method='nearest').to_pandas().sum()

0.0

In [38]:

#1C. Export emisi per crops dari kategori luc crops ke netcdf
luc_crops_prop_crop_netcdf = xarray.Dataset(
coords={
        "time": pd.to_datetime(["2020-01-01"]),
        "latitude": gfrac.coords["latitude"].to_numpy(),
        "longitude": gfrac.coords["longitude"].to_numpy()
    })

coords = ("time", "latitude", "longitude")
data_vars = {
    ngfbc: (coords, luc_prop_crop[i]) for i, ngfbc in enumerate(gfrac_ngfbfc)
}
luc_crops_prop_crop_netcdf = luc_crops_prop_crop_netcdf.assign(data_vars)

In [39]:
n = 0
luc_crops_prop_crop_netcdf['grass'].isel(time=0).to_numpy().sum()

9.610725

In [22]:
luc_crops_prop_crop_netcdf.to_netcdf("D:/kerja/asisten riset/code v/step 3/GbGlct/dataset/AM3/crop_emission_from_luc_crops_AM3.NC", mode="w", format="NETCDF4")

## Agri to agri

In [40]:
emission_CO2_netcdf_AM2['Agricultural Transition Emission'].sel(latitude=y[n], longitude=x[n], method='nearest').to_pandas()

time
1975-01-01    0.000000
1980-01-01    0.000000
1985-01-01    0.000003
dtype: float32

In [24]:
emission_CO2_netcdf_AM2['Agricultural Transition Emission'].isel(time=1).to_numpy().sum()

1.005606

In [41]:
emission_Agricultural = emission_CO2_netcdf_AM2['Agricultural Transition Emission'].isel(time=2).to_numpy()

In [42]:
#2C Distribusi emisi sesuai delta
agri_to_agri_prop_crop = np.zeros((17, 1, 2160, 4320))

for i, crop in enumerate(gfrac_list):
    agri_to_agri_prop_crop[i][0] = multiple(emission_Agricultural, proporsi[i][0])

In [43]:
# np.count_nonzero(np.isinf(agri_to_agri_prop_crop))
agri_to_agri_prop_crop = np.nan_to_num(agri_to_agri_prop_crop)
agri_to_agri_prop_crop[0][0].sum()

0.47630453983547094

In [99]:
agri_to_agri_prop_crop_netcdf = xarray.Dataset({
    "agri_to_agri_prop":(["NGFBFC","time","latitude", "longitude"], agri_to_agri_prop_crop)
},coords={
        "time": pd.to_datetime(["2020-01-01"]),
        "NGFBFC" : gfrac_ngfbfc,
        "latitude": gfrac.coords["latitude"].to_numpy(),
        "longitude": gfrac.coords["longitude"].to_numpy()
})

In [44]:
#2D Export emisi per crops dari kategori agri2agri ke netcdf
agri2agri_prop_crop_netcdf = xarray.Dataset(
coords={
        "time": pd.to_datetime(["2020-01-01"]),
        "latitude": gfrac.coords["latitude"].to_numpy(),
        "longitude": gfrac.coords["longitude"].to_numpy()
    })

coords = ("time", "latitude", "longitude")
data_vars = {
    ngfbc: (coords, agri_to_agri_prop_crop[i]) for i, ngfbc in enumerate(gfrac_ngfbfc)
}
agri2agri_prop_crop_netcdf = agri2agri_prop_crop_netcdf.assign(data_vars)
agri2agri_prop_crop_netcdf.to_netcdf("D:/kerja/asisten riset/code v/step 3/GbGlct/dataset/AM3/crop_emission_from_agri2agri_AM3.NC", mode="w", format="NETCDF4")

In [77]:
n = 5
agri2agri_prop_crop_netcdf['grass'].isel(time=0).sel(latitude=2580, longitude=395, method='nearest').to_pandas()

array(0., dtype=float32)

In [45]:
emisi_total = np.zeros((17, 1, 2160, 4320))

for i, crop in enumerate(gfrac_list):
    temp = agri_to_agri_prop_crop[i][0] + luc_prop_crop[i][0]
    emisi_total[i][0] = np.where(np.isfinite(temp), temp, 0)

In [48]:
emisi_total[0][0].sum(), agri_to_agri_prop_crop[0][0].sum(), luc_prop_crop[0][0].sum()

(10.087032328483524, 0.47630453983547094, 9.610725)

In [50]:
agri_to_agri_prop_crop[0][0].sum() + luc_prop_crop[0][0].sum()

10.087029942667503

In [42]:
emisi_total_netcdf = xarray.Dataset(
coords={
        "time": pd.to_datetime(["2020-01-01"]),
        "latitude": gfrac.coords["latitude"].to_numpy(),
        "longitude": gfrac.coords["longitude"].to_numpy()
    })

coords = ("time", "latitude", "longitude")
data_vars = {
    ngfbc: (coords, emisi_total[i]) for i, ngfbc in enumerate(gfrac_ngfbfc)
}
emisi_total_netcdf = agri2agri_prop_crop_netcdf.assign(data_vars)

In [48]:
emisi_total_netcdf.isel(time=0).sel(latitude=y[n], longitude=x[n], method='nearest').to_pandas().sum()

1.3161383318749387e-06

In [ ]:
country_code = pd.read_excel("D:/kerja/asisten riset/vol/milkunC/achaidir/LUH2 2022/ISO-3166-Country-Code_Final.xlsx")
luh_static = xarray.open_dataset("D:/kerja/asisten riset/code v/step 3/GbGlct/gpw_v4_national_identifier_grid_rev11_5_min_finall.nc", engine="netcdf4")

In [ ]:
ccode_iso = list(country_code['country-code'])
cname_iso = list(country_code['IMAGE Region Name'])

ccode_dict = {}
for i, ccode in enumerate(ccode_iso):
    ccode_dict[ccode] = cname_iso[i]

ccode_worldwide_int = luh_static['ccode'].to_numpy().astype('int64')

ccode_convert = np.zeros((2160, 4320), dtype="<U32") #rubah ke <U14

for i in range(2160):
    for j in range(4320):
        if (ccode_worldwide_int[i][j] in ccode_dict.keys()):
            ccode_convert[i][j] = ccode_dict[ccode_worldwide_int[i][j]]
        else:
            ccode_convert[i][j] = "ocean"
            
region = xarray.Dataset({"IMAGE Region Name": (["latitude", "longitude"], ccode_convert)},
                         coords={ "longitude": gfrac.coords["longitude"].to_numpy(), "latitude": gfrac.coords["latitude"].to_numpy()})

In [ ]:
emisi_total_am3_netcdf = xarray.Dataset(
coords={
        "time": pd.to_datetime(["2020-01-01"]),
        "latitude": gfrac.coords["latitude"].to_numpy(),
        "longitude": gfrac.coords["longitude"].to_numpy(),
    })

coords = ("time", "latitude", "longitude")
data_vars = {
    ngfbc: (coords, emisi_total[i]) for i, ngfbc in enumerate(gfrac_ngfbfc)
}
emisi_total_am3_netcdf = emisi_total_am3_netcdf.assign(data_vars)
GFRAC_IMAGERegionName = emisi_total_am3_netcdf.assign_coords(IMAGE_Region_Name=region["IMAGE Region Name"])

In [ ]:
GFRAC_IMAGERegionName.to_netcdf("D:/kerja/asisten riset/code v/step 3/GbGlct/dataset/AM3/emisi_total_AM3.NC", mode="w", format="NETCDF4")

C:\Users\HEFRYA~1\AppData\Local\Temp/ipykernel_312/3303792535.py:1: SerializationWarning: coordinate 'IMAGE Region Name' has a space in its name, which means it cannot be marked as a coordinate on disk and will be saved as a data variable instead
  emisi_total_am3_netcdf.to_netcdf("D:/kerja/asisten riset/code v/step 3/GbGlct/dataset/AM3/emisi_total_AM3.NC", mode="w", format="NETCDF4")


In [ ]:
import xarray

emisi_total_am3_netcdf = xarray.open_dataset("D:/kerja/asisten riset/code v/step 3/GbGlct/dataset/AM3/emisi_total_AM3.NC", engine="netcdf4")

<xarray.Dataset>
Dimensions:                           (longitude: 4320, latitude: 2160, time: 1)
Coordinates:
  * longitude                         (longitude) float64 -180.0 ... 180.0
  * latitude                          (latitude) float64 89.96 89.88 ... -89.96
  * time                              (time) datetime64[ns] 2020-01-01
Data variables: (12/18)
    IMAGE Region Name                 (latitude, longitude) <U17 ...
    grass                             (time, latitude, longitude) float64 ...
    Wheat                             (time, latitude, longitude) float64 ...
    Rice                              (time, latitude, longitude) float64 ...
    Maize                             (time, latitude, longitude) float64 ...
    Tropical cereals                  (time, latitude, longitude) float64 ...
    ...                                ...
    Tropical roots & tubers           (time, latitude, longitude) float64 ...
    Sugar crops                       (time, latitude, longitude) float64 ...
    Oil & palm fruit                  (time, latitude, longitude) float64 ...
    Vegetables & fruits               (time, latitude, longitude) float64 ...
    Other non-food & luxury & spices  (time, latitude, longitude) float64 ...
    Plant based fibres                (time, latitude, longitude) float64 ...

In [ ]:
df = GFRAC_IMAGERegionName.isel(time=0).to_dataframe()
table = pd.pivot_table(df, values=gfrac_ngfbfc, index=["region_name"], columns=['time'], aggfunc="sum", fill_value=0)

C:\Users\HEFRYA~1\AppData\Local\Temp/ipykernel_17388/1129014085.py:3: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df_index = table.stack(level=0)


time,region_name,type,2020
0,Brazil,Maize,0.417519
1,Brazil,Oil & palm fruit,0.000319
2,Brazil,Other non-food & luxury & spices,0.116713
3,Brazil,Other temperate cereals,0.013949
4,Brazil,Plant based fibres,0.194890
...,...,...,...
471,ocean,Tropical oil crops,0.003549
472,ocean,Tropical roots & tubers,0.000670
473,ocean,Vegetables & fruits,0.001246
474,ocean,Wheat,0.000560


In [ ]:
df_index = table.stack(level=0, future_stack=True)
data = pd.to_datetime(df_index.columns, format='%d/%m/%Y %H.%M.%S')
df_index.columns = data.year
df_index = df_index.reset_index()
df_index.rename(columns={'level_1': "type"}, inplace=True)
df_index = df_index.replace([np.inf, -np.inf], np.nan)
df_index = df_index.fillna(0)
df_index.to_excel("D:/kerja/asisten riset/code v/step 3/GbGlct/dataset/AM3/Emisitotal_RegionType_AM3.xlsx", index=False)